# Week 2: Odometry-based Navigation
## Learning Objectives
* Understand how odometry data relates to the *position* and *motion* of your Robot
* Issue commands to the TurtleBot3 robot to make it move
* Write python scripts to work with odometry data and perform navigation with your robot (dead-reckoning navigation)

## Introduction

In a ROS robot, basic navigation is achieved by sending velocity commands to the wheel motors and by using **odometry** to approximate where the robot is. 

## Launching the Robot Simulation
1. Open the **Tools** menu in the top left hand corner of your workspace and select **Gazebo** to open up a *Gazebo Simulation Instance*
* Enter the **Tools** menu again and select **Shell** to open up a *Linux terminal window*
* In the terminal enter the following command to launch a TurtleBot3 simulation model in an empty world:

    `$ roslaunch turtlebot3_gazebo turtlebot3_empty_world.launch`


**NOTE:** *A command starting with a `$` indicates a complete command that can be typed (or copied and pasted) directly into a terminal.  Enter all the text except for the `$` symbol*

In the Gazebo simulation window, you should see a *TurtleBot3 Waffle* in empty space:
<img src="figures/Gazebo_TB3_EmptyWorld.PNG">

## Basic Navigation
As you will have already learnt from the "Introduction to the TurtleBot3 Waffle" section of your labsheet, the robot has the following sensors and actuators on-board to allow the robot to navigate:
* Two independently controlled wheel motors (*differential drive*)
* An Inertial Measurement Unit (IMU) to detect motion/orientation
* A 360&deg; laser displacement sensor (LIDAR) to detect its environment

Two types of speed command can be issued to any robot:
* **Linear speed**: The speed at which the robot moves forwards or backwards in one of the robot axes
* **Angular speed**: The speed at which the robot rotates about one of its axes

The TurtleBot3's principal axes are defined as follows:
<img src="figures/TB3 Axes.png">

Because the TurtleBot3 robot has a *differential drive* configuration, it can only move **linearly** in the *roll (x)* axis.  In order to move to the left or right, it must first rotate to face the desired direction before moving forward.  Due to the configuration of it's drive motors, the robot can only **rotate** about it's *yaw (z)* axis.

In the cell below use a **ROS command** to list of all the active topics that are currently running now that the simulation is active (you learnt how to do this last week).

**Note:**  *Using the <code>!</code> symbol in this notebook allows us to issue commands to the underlying operating system, leave it where it is and type in your command after it.  This is the same as running the command (without the <code>!</code> symbol) in a terminal. With your cursor in the cell, press the <b>"Run"</b> button: <img src="figures/run_cell.PNG"> or press `CTRL + ENTER`  to run the cell:*

In [ ]:
!

Which topic in the list do you think could be used to control the movement of the robot?  Use the `rostopic info` command on the topic in the cell below to find out more about it:

In [ ]:
!

The topic you have identified should use a message of type `geometry_msgs/Twist`. You will have to send messages in this format to this topic in order to make the robot move. In the following cell, use the `rosmsg` command to find out more about the message format:

In [ ]:
!

As we learnt above, the TurtleBot3 can only generate **linear velocity** in the *x axis*, and can only implement **angular velocity** in the *z axis*.  As a result, only velocity commands issued to `linear.x` or `angular.z` will have any effect.

### Exercise 1: Exploring Odometry Data

Another topic that should have appeared in the list of all active topics above is `/odom`.  This topic contains **Odomerty** data, which is also essential for robot navigation. 

1. In **a new terminal window** use the `rostopic echo` command to display the odometry data currently being published by our simulated robot:

    `$ rostopic echo /odom`


2. In **another new terminal window** launch the *keyboard_teleop* node as you did last week:

    `$ roslaunch turtlebot3_teleop turtlebot3_teleop_key.launch`


3. With the `rostopic echo` terminal and the *Gazebo Simulation* windows visible, but the `turtlebot3_teleop_key` terminal window active observe how the odometry data changes as the robot moves around in its simulated world

4. Once done, press `CTRL + C` in the first webshell to stop the `rostopic echo` node (leave the keyboard teleop node running)

5. Next, with the robot stationary, run a command to read **a single message** from the `/odom` topic (you learnt how to do this last week).  Find the section of the odometry message that contains the information about the robot's *position* and *orientation*.  Assign the values that you obtain from this to the variables in the cell below.  Click the *"Run"* button to run the cell, the values you entered should be printed below the cell.

In [ ]:
from support_matls.odometry import Odometry

start = Odometry()

## input your values here: ##
start.position.x = 
start.position.y = 
start.position.z = 

start.orientation.x = 
start.orientation.y = 
start.orientation.z = 
start.orientation.w = 

print(start)

6. Move the robot around a bit using the *turtlebot3_teleop* node
7. Get some new data from the `/odom` topic by running the same `echo` command again
8. Input this new data into the cell below in the same way as before, and run this cell as well.  The values you entered should be printed, as well as a summary of the difference between the odometry data before and after the movement.

In [ ]:
end = Odometry()

## input your values below: ##
end.position.x = 
end.position.y = 
end.position.z = 

end.orientation.x = 
end.orientation.y = 
end.orientation.z = 
end.orientation.w = 
##### end of user input ######

print(end)

from tf.transformations import euler_from_quaternion

(roll1, pitch1, yaw1) = euler_from_quaternion(
                            [start.orientation.x, start.orientation.y,
                             start.orientation.z, start.orientation.w], 
                            'sxyz')

(roll2, pitch2, yaw2) = euler_from_quaternion(
                            [end.orientation.x, end.orientation.y,
                             end.orientation.z, end.orientation.w], 
                            'sxyz')

from support_matls.odometry import delta

df = delta(start, (roll1, pitch1, yaw1), 
          end, (roll2, pitch2, yaw2))

df

### Odometry
The Odometry topic contains four base elements:
1. header
2. child_frame_id
3. pose
4. twist  
---
    header:
      seq: 21878
      stamp:
        secs: 729
        nsecs: 433000000
      frame_id: "odom"
    child_frame_id: "base_footprint"
    pose:
      pose:
        position:
          x: 0.000151308769464
          y: 0.000432896574121
          z: -0.00100739934145
        orientation:
          x: -3.82053264254e-06
          y: 0.00158964733229
          z: 0.00135862129503
          w: 0.999997813575
      covariance: [1e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1e-05, 0.0,0.0, 0.0, 0.0, 0.0, 0.0, 1000000000000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1000000000000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1000000000000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.001]
    twist:
      twist:
        linear:
          x: 7.15591876568e-07
          y: 1.40367536648e-06
          z: 0.0
        angular:
          x: 0.0
          y: 0.0
          z: 9.04325760963e-06
      covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

---

**pose** tells us the position and orientation of the robot relative to an arbitrary starting point (typically the zero point is set when the robot starts up, but this can also be zeroed manually).  This information is based on: 
1. Data from a Gyroscope, which is part of the Inertial Measurement Unit (*IMU*) onboard the Open-CV board 
1. Data from each of the wheel encoders
1. An **estimation** of the distance travelled from a pre-defined reference point (dead-reckoning)

Pose data is important for determining the movement of our robot, and from this we can estimate its position and orientation in 3-dimensional space.  *Orientation* is expressed in units of [Quaternions](https://en.wikipedia.org/wiki/Quaternion), and needs to be converted into angles about the principal axes.  Fortunately, there are functions within the ROS `tf` library to do that for us, and the usage of this was illustrated in the exercise above:
```python
from tf.transformations import euler_from_quaternion

(roll, pitch, yaw) = euler_from_quaternion([orientation.x, 
                       orientation.y, orientation.z, orientation.w], 
                       'sxyz')
```
Our TurtleBot3 robot can only move in a 2D plane, and so actually, the pose of the robot can be fully represented by <code>(x,y,&#952;<sub>z</sub>)</code>, where `x` and `y` are the 2D coordinates of the robot and <code>&#952;<sub>z</sub></code> is the angle of the robot about the `z` axis (the `yaw`).  You should have noticed this from the output of the cell above, where the **linear_z**, **theta_x** and **theta_y** values in the **delta** column should all have read `0.0`.

**twist** tells us the current linear and angular velocities of the robot, and this data comes directly from the wheel encoders.

All this data is defined in terms of the principal axes illustrated in the figure above.

{FIGURE TO ILLUSTRATE ROBOT POSE}

### Exercise 2: Moving a robot with `rostopic` in the terminal

**NOTE:** *Before starting this exercise, you will need to stop the **turtlebot3_teleop** node.  Navigate to the terminal window where this node is running and enter `CTRL + C` to stop it.*

We can use the `rostopic pub` command to publish data to a topic from within a terminal by using the command in the following way:

`rostopic pub [topic_name] [message_type] [data]`
    

As we discovered earlier, the <code>/cmd_vel</code> topic is expecting **linear** and **angular** data, each with an <b>x</b>, <b>y</b> and <b>z</b> component.  We can get further help with formatting this message by using the autocomplete functionality within the terminal (as you discovered last week).  **Type** the following into an inactive terminal window (copying and pasting will not work here):

`rostopic pub /cmd_vel geometry_msgs/Twist [SPACE] [TAB] [TAB]`

1. Use this to help you enter velocity commands in the terminal.  Enter values to make the robot rotate on the spot and note down the command that you used in the cell below:

2. Enter `CTRL + C` in the terminal to stop the message from being published
3. Next, enter a command in the terminal to make the robot move in a circle and, again, note down the command that you used in the cell below:

4. Enter `CTRL + C` in the terminal to stop the message from being published
5. How do we stop the TurtleBot?  Note down the command that you used in the cell below:

6. Enter `CTRL + C` in the terminal to stop the message from being published

### Exercise 3: Creating a python node to process Odometry data

Last week you learnt how to create a package and build simple nodes in Python to publish and subscribe to messages on a topic.  You can view the worksheet for last weeks lab session [here](past_weeks/COM2009_WK01/wk01%20-%20ROS%20Concepts%201.ipynb)

1. In the same way as last week, you will need to create a package, this time called **nav_exercise**, which depends on the `rospy`, `nav_msgs`, `geometry_msgs` and `tf` libraries.  Use the `catkin_create_pkg` tool as you did last week.  *Remember to ensure that you are located in the **~/catkin_ws/src/** directory before you do*:
    
    `$ cd ~/catkin_ws/src/`
    
    
2. Navigate to the **src** folder within your **nav_exercise** package using the `cd` command
3. The **subscriber.py** file that you used last week is available on github, and you can use this as a template for creating your odometry subscriber.  Download the file into the **~/catkin_ws/src/nav_exercise/src** folder by using the following command once you are located in the right place:

    `$ wget https://raw.githubusercontent.com/tom-howard/COM2009_WK01/master/support_matls/subscriber.py`
  
  
4. Next, use the linux `mv` command to rename (or *move*) the **subscriber.py** file to **odom_subscriber.py**: 

    `$ mv subscriber.py odom_subscriber.py`
    
    
5. In the same way as last week, make this file executable using the `chmod` command
6. Open up the *IDE*, navigate to the **odom_subscriber.py** file and edit it to subscribe and print odometry data to the console.  Your python node should convert the raw odometry data to a <code>(x,y,&#952;<sub>z</sub>)</code> format using the `euler_from_quaternion` function from the `tf.transformations` library as illustrated earlier
5. Launch your node using `rosrun` and observe the changes to the odometry data whilst moving the robot around using the *keyboard_teleop* node

**At this point, show your work to a demonstrator**

### Exercise 4: Creating a python node to make the TurtleBot3 move

You will now create another node to control the motion of your TurtleBot3 robot bu publishing messages to the `/cmd_vel` topic.  You created publisher node last week, and you can use this as a template to get you started.

1. First, close down the `turtlebot3_teleop_key` node by navigating to the terminal where it is active, and entering `CTRL + C`.  You can then close down this terminal window if you want to.
2. Then, also close down your **odom_subscriber.py** node in the same way
3. Ensure that you are still located within the **src** folder of your **nav_exercise** package.  You could use `$ pwd` to check your current working directory, and the output should look like this:
        /home/user/catkin_ws/src/nav_exercise/src
    If you aren't located here, then navigate to this directory using `cd`
4. The **publisher.py** file that you used last week is also available on github.  Download the file into the **nav_exercise/src** folder by using the following command in the right location:

    `$ wget https://raw.githubusercontent.com/tom-howard/COM2009_WK01/master/support_matls/publisher.py`
  
  
5. As above, use the `mv` command to rename the file to **move_circle.py**    
6. Again, make sure **this** file is also **executable** using the `chmod` command
7. Open up the *IDE*, navigate to the new file and edit it.  
    1. The goal is to make your TurtleBot3 move in a circle, with a path radius of approximately 0.5m
    2. The python node needs to publish `Twist` messages to the `/cmd_vel` topic in order to make the TurtleBot move.  Have a look at a usage example [here](support_matls/Wk2%20Ex4%20TwistUsageExample.ipynb)  
    3. For our robots, the maximum linear velocity (`linear.x`) is 0.26m/s, and the maximum angular velocity (`angular.z`) is 1.82rad/s 
    4. Make sure that you program your `shutdown_function()` correctly so that the robot stops moving when the node is shutdown (via `CTRL + C` from the terminal)
8. Create a launch file for your **move_circle.py** and **odom_subscriber.py** nodes as we did last week, and launch them using the `roslaunch` command

**Once done, show your work to a demonstrator**

### Exercise 5: Make your robot follow a *Square* motion profile

1. Close down your python nodes from Exercise 4 by using `CTRL + C`
1. Create a copy of the **move_circle.py** node that you created in Execise 4 using the `cp` command and call it **move_square.py** 
1. Modify the code in the *IDE* to make your robot follow a square motion profile of 1m x 1m dimensions
1. Create a new launch file to launch the **move_square.py** and **odom_subscriber.py** files and launch them using the `roslaunch` command

**Once done, show your work to a demonstrator**

## Summary of Week 2

This is the end of the Week 2 lab tasks.  Next week you will implement the motion controller package that you have built here on an actual robot, and also look at more advanced methods of robot navigation using the *ROS Navigation Stack* and **SLAM**.

* In this session we have explored the odometry data that is published to the `/odom` topic by a robot
    * This tells us where in physical space our robot is located and oriented, which can be used for *dead-reckoning navigation*
    * The odometry data also contains *Twist* information, which tells us the current linear and angular velocities of our robot in terms of its 3 principal axes
* We have also used the `cmd_vel` topic to publish velocity commands to our robot to make it move 
* From the exercises, we have learnt that navigation based on odometry alone is not easy, and can quickly lead to accumulated error
* As a result, more advanced navigation techniques are required in order to more accurately perform navigation with a robot

# Saving, exiting and downloading a copy of your work

1. You should now save this notebook using the **Save and Checkpoint** button in the top left of this Jupyter Notebook window: <img src="figures/pynb_save_button.PNG">
2. Then click the **Save your ROSject** button in the top right hand side of the browser window
3. Once done, click on the **Manage your ROSjects** button to return to your main dashboard page for the ROS Development Studio: <img src="figures/ROSDS_buttons.PNG">
4. When in your dashboard area, click on the download button for this weeks project to download a copy of all the work you have done today to a zip archive for your own records (you may need to use some of this next week): <img src="figures/dashboard_buttons.PNG">

# Acknowledgements

This notebook was created by Dr Tom Howard (The University of Sheffield, UK) based on pre-existing course material published by [The Construct](https://www.theconstructsim.com/) and [the Official ROS Tutorials](http://wiki.ros.org/ROS/Tutorials)

Tom Howard  
The University of Sheffield  
Last Updated: 10/01/2020